# Sentence Similarity

In [1]:
from keras.preprocessing.text import Tokenizer
import numpy as np

Using TensorFlow backend.


In [2]:
'''take out the data from the file and put t in a list '''
input_text=[]
for line in open('list_of_sentences','r'):
    input_text.append(line.rstrip())
    
input_text

['good morning',
 'how are you doing',
 'the weather is awesome today',
 'samsung',
 'good afternoon',
 'baseball is played in the USA',
 'there is a thunderstorm',
 'are you doing good',
 'The polar regions are melting',
 'apple',
 'nokia',
 'cricket is a fun game',
 'the climate change is a problem']

In [3]:
'''Tokenize the list of sentences'''
# tokenize the inputs
tokenizer_inputs = Tokenizer()
tokenizer_inputs.fit_on_texts(input_text)
#input_sequences = tokenizer_inputs.texts_to_sequences(input_texts)


# get the word to index mapping for input language
word2idx_inputs = tokenizer_inputs.word_index
num_words=len(word2idx_inputs)
num_words
word2idx_inputs

{'is': 1,
 'the': 2,
 'good': 3,
 'are': 4,
 'a': 5,
 'you': 6,
 'doing': 7,
 'morning': 8,
 'how': 9,
 'weather': 10,
 'awesome': 11,
 'today': 12,
 'samsung': 13,
 'afternoon': 14,
 'baseball': 15,
 'played': 16,
 'in': 17,
 'usa': 18,
 'there': 19,
 'thunderstorm': 20,
 'polar': 21,
 'regions': 22,
 'melting': 23,
 'apple': 24,
 'nokia': 25,
 'cricket': 26,
 'fun': 27,
 'game': 28,
 'climate': 29,
 'change': 30,
 'problem': 31}

In [4]:
'''used glove pretrained word embedding'''
# store all the pre-trained word vectors
EMBEDDING_DIM = 300
print('Loading word vectors...')
word2vec = {}
with open('glove.6B.%sd.txt' % EMBEDDING_DIM) as f:
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec
print('Found %s word vectors.' % len(word2vec))




# prepare embedding matrix
print('Filling pre-trained embeddings...')
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word2idx_inputs.items():
    embedding_vector = word2vec.get(word)
    if embedding_vector is not None:
      # words not found in embedding index will be all zeros.
      embedding_matrix[i-1] = embedding_vector 

Loading word vectors...
Found 400000 word vectors.
Filling pre-trained embeddings...


In [6]:
len(embedding_matrix)

31

In [22]:
'''calculate the average of embedding for each sentence'''
sentence_embedding={}
for sentence in  input_text:
    overall_value_of_sentence = np.zeros((1,300))
    no_of_words=0
    for word in sentence.lower().split():
        overall_value_of_sentence +=embedding_matrix[word2idx_inputs[word]-1]
        no_of_words +=1  
    print(sentence)
    print(no_of_words)
    sentence_embedding[sentence]=overall_value_of_sentence/no_of_words         

good morning
2
how are you doing
4
the weather is awesome today
5
samsung
1
good afternoon
2
baseball is played in the USA
6
there is a thunderstorm
4
are you doing good
4
The polar regions are melting
5
apple
1
nokia
1
cricket is a fun game
5
the climate change is a problem
6


In [29]:
'''helps in calculating the cosine similarity between pair of vectors'''
from sklearn.metrics.pairwise import cosine_similarity


In [39]:
'''take out the cosine similarity for each sentence with all the other sentence in our corpus'''
dic_of_similarity={}

for i in range(len(input_text)):
    similarity = []
    embedding1 = sentence_embedding[input_text[i]]
    for j in range(len(input_text)):
        embedding2 = sentence_embedding[input_text[j]]
        sim = cosine_similarity(embedding1, embedding2)[0][0]
        similarity.append({input_text[j]:sim})
    dic_of_similarity[input_text[i]]=similarity
        

In [77]:
'''take out the list of similar senteces which I assumed on maching more than equal to 73%'''
overall_group = []
for i in range(len(input_text)):
    group_text=[]
    for j in range(len(input_text)):
        val=dic_of_similarity[input_text[i]][j]
        if val[input_text[j]]*100>=73.0:
            for key in val.keys():
                group_text.append(key)
    overall_group.append(group_text)

In [87]:
overall_group

[['good morning', 'good afternoon'],
 ['how are you doing', 'are you doing good'],
 ['the weather is awesome today',
  'there is a thunderstorm',
  'the climate change is a problem'],
 ['samsung'],
 ['good morning', 'good afternoon'],
 ['baseball is played in the USA', 'cricket is a fun game'],
 ['the weather is awesome today',
  'there is a thunderstorm',
  'the climate change is a problem'],
 ['how are you doing', 'are you doing good'],
 ['The polar regions are melting'],
 ['apple'],
 ['nokia'],
 ['baseball is played in the USA', 'cricket is a fun game'],
 ['the weather is awesome today',
  'there is a thunderstorm',
  'the climate change is a problem']]

In [95]:
'''take out the unique list of similar senteces and keep it in final_result'''
final_result=[]
for entry in overall_group:
    if entry not in final_result:
        final_result.append(entry)



In [96]:
'''return the final result'''
final_result

[['good morning', 'good afternoon'],
 ['how are you doing', 'are you doing good'],
 ['the weather is awesome today',
  'there is a thunderstorm',
  'the climate change is a problem'],
 ['samsung'],
 ['baseball is played in the USA', 'cricket is a fun game'],
 ['The polar regions are melting'],
 ['apple'],
 ['nokia']]